In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
style.use('ggplot')

df = pd.read_excel('PL_2019-2020.xlsx')
df

,SEASON_CD,SEASON_NAME,PLOT_CLUSTER_CD,CLUSTER_NAME,PLOT_VILLAGE_CD,VILLAGE_NAME,FARMER_ID,F_NAME,PLOT_NO,AG_FLAG,...,DIVERTED_AREA,DISPOSAL_AREA,BOOKED_AREA,NON_BOOKED_AREA,SUPPLIED_TONNS_1,YIELD,AGE_IN_DAYS,GBL_PLANT_ID,GBL_KHATE_ID,GBL_PLOT_ID
0,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,13.0,NaN,...,0.0,0.0,4.0,0.0,187.763,46.941,426,914297,11,12
1,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,4.0,NaN,...,0.0,0.0,5.0,0.0,99.020,19.804,361,984477,10,9
2,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,5.0,NaN,...,0.0,0.0,5.0,0.0,40.679,8.136,347,985669,10,10
3,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100003,SATYAPPA SHASAPPA KAMADINNI,7.0,NaN,...,6.0,0.0,6.0,0.0,0.000,NaN,524,903118,19,26
4,25,2019-2020,1,FACTORY EAST,2,AKKIMARADI,100004,VENKAPPA SHASAPPA KAMADINNI,1.0,NaN,...,0.0,0.0,5.0,0.0,57.281,11.456,387,917920,20,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,365911,KARIYAPPA HUCHAPPA HUGGI,2.0,NaN,...,0.0,2.0,0.0,2.0,0.000,NaN,65,979755,238108,327730
53421,25,2019-2020,28,LOKAPUR,1453,KAMATAGI,369510,SAKRAPPA YALLAPPA HAVOJI,1.0,NaN,...,0.0,0.0,1.0,0.0,21.206,21.206,404,1011696,246928,337841
53422,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,1.0,NaN,...,0.0,0.0,1.0,0.0,23.849,23.849,386,1018000,250026,341175
53423,25,2019-2020,28,LOKAPUR,1468,SHIROL (GADAG),371252,SAVITRI MUDAKANAGOUDA TIRAKANAGOUDR,2.0,NaN,...,0.0,0.0,1.0,0.0,21.810,21.810,392,1018001,250026,341176


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report, precision_score,recall_score,f1_score
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
df.CV_NAME.value_counts()

86032           37802
91010            9001
M 265            4305
SNK-09293         922
PI 00-1110        434
CO VSI 08005      311
CO 92005           95
CO-2001-13         91
MS-10001           88
OTHERS             70
8011               63
671                59
SNK-09227          43
CO-09004           41
CO SNK 07680       38
CO62175            28
CO-SNK-632         11
MIX                 6
94012               5
CO-2001-15          3
CO-15017            3
CO-14031            2
92020               2
CO-2012-293         1
EC-14006            1
Name: CV_NAME, dtype: int64

In [10]:
df1=df[df.CV_NAME == '86032']
X1=df1[["CLUSTER_NAME","PLANT_AREA","CROP_TYPE_NAME","AGE_IN_DAYS"]].copy()
Y1=df1["DIVERSION_CENTER_NAME"].copy()
X1,Y1
Y1.value_counts()

GSM_HARVEST                          16917
FAILED                                5609
NIRANI SUGARS                         2542
SAI PRIYA SUGARS HIPPARAGI            2218
UGAR SUGARS                           2166
PRABHULINGESHWAR SUGARS               1500
FLOOD                                  989
SHIVASHAKTI SUGARS YADRANV             827
REMOVED                                690
RENUKA SUGARS KOKATNUR                 684
INDIA CANE POWER LTD. UTTUR            668
JAMAKHANDI SUGARS LTD                  518
SATISH SUGARS, HUNSHYAL                502
ATHANI SSK. KEMPAWAD                   386
SELF_SEEDS                             260
RENUKA  SUGARS RAIBAG                  216
GSM_SEEDS                              173
JAGGERY                                164
RENUKA SUGARS KOLAVI                   160
GEMS SUGARS                            121
KRISHNA SSK HALYAL                     116
RANNA CO-OP SUGARS                      90
BILAGI, SUGARS, BADAGANDI               51
NANDI SSK  

In [4]:
for i in Y1.index:
    Y1[i]=(1 if (Y1[i]=="GSM_HARVEST") else 0)
Y1_new=Y1.astype('int')
Y1_new

3        0
5        1
6        0
7        0
8        1
        ..
53418    1
53419    0
53420    0
53421    1
53424    1
Name: DIVERSION_CENTER_NAME, Length: 37802, dtype: int32

In [6]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

features = ["CLUSTER_NAME","CROP_TYPE_NAME"]
one = OneHotEncoder(sparse=False)
transformer = ColumnTransformer([("one",
                                  one,
                                  features)],
                                  remainder="passthrough")
X1_trans = transformer.fit_transform(X1)
X1_new = pd.DataFrame(X1_trans)
X1_new,Y1_new.dtypes

(        0    1    2    3    4    5    6    7    8    9   ...   15   16   17  \
 0      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 1      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 3      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 4      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 37797  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 37798  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 37799  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 37800  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 37801  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
 
         18   19   20   21   22   23  

In [7]:
X1train,X1test,Y1train,Y1test =train_test_split(X1_new,Y1_new,test_size=0.2)

In [12]:
hgbc = HistGradientBoostingClassifier()
hgbc.fit(X1train,Y1train)
hgbc_score=hgbc.score(X1test,Y1test)
hgbc_score

0.8427456685623594

In [8]:
xgboost = XGBClassifier()
xgboost.fit(X1train,Y1train)
xgboost_score=xgboost.score(X1test,Y1test)
catboost = CatBoostClassifier()
catboost.fit(X1train,Y1train)
catboost_score=catboost.score(X1test,Y1test)
lgbm = LGBMClassifier()
lgbm.fit(X1train,Y1train)
lgbm_score=lgbm.score(X1test,Y1test)
xgboost_score,catboost_score,lgbm_score

C:\Users\91720\.conda\envs\env\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:14:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.044172
0:	learn: 0.6492300	total: 172ms	remaining: 2m 51s
1:	learn: 0.6149041	total: 183ms	remaining: 1m 31s
2:	learn: 0.5800602	total: 194ms	remaining: 1m 4s
3:	learn: 0.5545991	total: 205ms	remaining: 50.9s
4:	learn: 0.5308104	total: 214ms	remaining: 42.6s
5:	learn: 0.5118928	total: 224ms	remaining: 37.1s
6:	learn: 0.4965537	total: 234ms	remaining: 33.2s
7:	learn: 0.4826234	total: 244ms	remaining: 30.2s
8:	learn: 0.4716692	total: 254ms	remaining: 27.9s
9:	learn: 0.4631820	total: 264ms	remaining: 26.2s
10:	learn: 0.4541226	total: 274ms	remaining: 24.6s
11:	learn: 0.4470329	total: 284ms	remaining: 23.4s
12:	learn: 0.4412040	total: 294ms	remaining: 22.3s
13:	l

154:	learn: 0.3502095	total: 1.69s	remaining: 9.22s
155:	learn: 0.3500757	total: 1.7s	remaining: 9.21s
156:	learn: 0.3499717	total: 1.71s	remaining: 9.21s
157:	learn: 0.3498904	total: 1.73s	remaining: 9.2s
158:	learn: 0.3498113	total: 1.74s	remaining: 9.19s
159:	learn: 0.3496913	total: 1.75s	remaining: 9.18s
160:	learn: 0.3495769	total: 1.76s	remaining: 9.17s
161:	learn: 0.3494900	total: 1.77s	remaining: 9.15s
162:	learn: 0.3494157	total: 1.78s	remaining: 9.13s
163:	learn: 0.3493316	total: 1.79s	remaining: 9.12s
164:	learn: 0.3492769	total: 1.8s	remaining: 9.1s
165:	learn: 0.3491776	total: 1.81s	remaining: 9.08s
166:	learn: 0.3491584	total: 1.81s	remaining: 9.05s
167:	learn: 0.3491056	total: 1.82s	remaining: 9.04s
168:	learn: 0.3489820	total: 1.83s	remaining: 9.02s
169:	learn: 0.3489024	total: 1.84s	remaining: 9.01s
170:	learn: 0.3487884	total: 1.85s	remaining: 8.99s
171:	learn: 0.3486941	total: 1.86s	remaining: 8.97s
172:	learn: 0.3486153	total: 1.87s	remaining: 8.96s
173:	learn: 0.34

326:	learn: 0.3380007	total: 3.39s	remaining: 6.98s
327:	learn: 0.3379044	total: 3.4s	remaining: 6.97s
328:	learn: 0.3377834	total: 3.41s	remaining: 6.96s
329:	learn: 0.3377321	total: 3.42s	remaining: 6.95s
330:	learn: 0.3376853	total: 3.43s	remaining: 6.94s
331:	learn: 0.3376632	total: 3.44s	remaining: 6.92s
332:	learn: 0.3376217	total: 3.45s	remaining: 6.91s
333:	learn: 0.3375833	total: 3.46s	remaining: 6.9s
334:	learn: 0.3375604	total: 3.47s	remaining: 6.88s
335:	learn: 0.3375396	total: 3.48s	remaining: 6.87s
336:	learn: 0.3374461	total: 3.49s	remaining: 6.86s
337:	learn: 0.3373930	total: 3.5s	remaining: 6.84s
338:	learn: 0.3373447	total: 3.5s	remaining: 6.83s
339:	learn: 0.3372806	total: 3.51s	remaining: 6.82s
340:	learn: 0.3372509	total: 3.52s	remaining: 6.81s
341:	learn: 0.3372066	total: 3.53s	remaining: 6.8s
342:	learn: 0.3371560	total: 3.54s	remaining: 6.79s
343:	learn: 0.3371349	total: 3.55s	remaining: 6.78s
344:	learn: 0.3370808	total: 3.57s	remaining: 6.77s
345:	learn: 0.337

501:	learn: 0.3305759	total: 5.09s	remaining: 5.05s
502:	learn: 0.3305319	total: 5.1s	remaining: 5.04s
503:	learn: 0.3305206	total: 5.11s	remaining: 5.03s
504:	learn: 0.3304671	total: 5.12s	remaining: 5.02s
505:	learn: 0.3304229	total: 5.13s	remaining: 5.01s
506:	learn: 0.3303969	total: 5.14s	remaining: 5s
507:	learn: 0.3303816	total: 5.15s	remaining: 4.99s
508:	learn: 0.3303539	total: 5.16s	remaining: 4.98s
509:	learn: 0.3303213	total: 5.17s	remaining: 4.97s
510:	learn: 0.3302779	total: 5.18s	remaining: 4.96s
511:	learn: 0.3302451	total: 5.19s	remaining: 4.95s
512:	learn: 0.3302049	total: 5.2s	remaining: 4.94s
513:	learn: 0.3301302	total: 5.21s	remaining: 4.93s
514:	learn: 0.3300771	total: 5.22s	remaining: 4.92s
515:	learn: 0.3300614	total: 5.23s	remaining: 4.91s
516:	learn: 0.3300223	total: 5.24s	remaining: 4.89s
517:	learn: 0.3299925	total: 5.25s	remaining: 4.88s
518:	learn: 0.3299524	total: 5.26s	remaining: 4.87s
519:	learn: 0.3299194	total: 5.27s	remaining: 4.86s
520:	learn: 0.329

666:	learn: 0.3250129	total: 6.68s	remaining: 3.33s
667:	learn: 0.3249823	total: 6.69s	remaining: 3.33s
668:	learn: 0.3249540	total: 6.7s	remaining: 3.31s
669:	learn: 0.3249184	total: 6.71s	remaining: 3.3s
670:	learn: 0.3248546	total: 6.72s	remaining: 3.29s
671:	learn: 0.3248248	total: 6.73s	remaining: 3.29s
672:	learn: 0.3248096	total: 6.74s	remaining: 3.27s
673:	learn: 0.3247742	total: 6.75s	remaining: 3.27s
674:	learn: 0.3247551	total: 6.76s	remaining: 3.25s
675:	learn: 0.3247292	total: 6.77s	remaining: 3.25s
676:	learn: 0.3246879	total: 6.78s	remaining: 3.23s
677:	learn: 0.3246565	total: 6.79s	remaining: 3.22s
678:	learn: 0.3246363	total: 6.8s	remaining: 3.21s
679:	learn: 0.3246126	total: 6.81s	remaining: 3.21s
680:	learn: 0.3245892	total: 6.82s	remaining: 3.19s
681:	learn: 0.3245482	total: 6.83s	remaining: 3.18s
682:	learn: 0.3245048	total: 6.84s	remaining: 3.17s
683:	learn: 0.3244810	total: 6.85s	remaining: 3.16s
684:	learn: 0.3244547	total: 6.86s	remaining: 3.15s
685:	learn: 0.3

825:	learn: 0.3211987	total: 8.18s	remaining: 1.72s
826:	learn: 0.3211826	total: 8.19s	remaining: 1.71s
827:	learn: 0.3211277	total: 8.2s	remaining: 1.7s
828:	learn: 0.3211101	total: 8.21s	remaining: 1.69s
829:	learn: 0.3210871	total: 8.22s	remaining: 1.68s
830:	learn: 0.3210755	total: 8.22s	remaining: 1.67s
831:	learn: 0.3210526	total: 8.23s	remaining: 1.66s
832:	learn: 0.3210208	total: 8.24s	remaining: 1.65s
833:	learn: 0.3209992	total: 8.25s	remaining: 1.64s
834:	learn: 0.3209768	total: 8.26s	remaining: 1.63s
835:	learn: 0.3209664	total: 8.27s	remaining: 1.62s
836:	learn: 0.3209423	total: 8.28s	remaining: 1.61s
837:	learn: 0.3208928	total: 8.29s	remaining: 1.6s
838:	learn: 0.3208476	total: 8.3s	remaining: 1.59s
839:	learn: 0.3208392	total: 8.31s	remaining: 1.58s
840:	learn: 0.3208181	total: 8.32s	remaining: 1.57s
841:	learn: 0.3207838	total: 8.33s	remaining: 1.56s
842:	learn: 0.3207518	total: 8.33s	remaining: 1.55s
843:	learn: 0.3207221	total: 8.34s	remaining: 1.54s
844:	learn: 0.32

990:	learn: 0.3167852	total: 9.7s	remaining: 88.1ms
991:	learn: 0.3167353	total: 9.71s	remaining: 78.3ms
992:	learn: 0.3167127	total: 9.72s	remaining: 68.5ms
993:	learn: 0.3166764	total: 9.73s	remaining: 58.7ms
994:	learn: 0.3166410	total: 9.74s	remaining: 48.9ms
995:	learn: 0.3166194	total: 9.75s	remaining: 39.2ms
996:	learn: 0.3165872	total: 9.76s	remaining: 29.4ms
997:	learn: 0.3165772	total: 9.77s	remaining: 19.6ms
998:	learn: 0.3165582	total: 9.78s	remaining: 9.79ms
999:	learn: 0.3165415	total: 9.79s	remaining: 0us


(0.839703742891152, 0.8423488956487237, 0.8424811532866023)

In [8]:

rc = RidgeClassifier()
svc = SVC()
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
rc.fit(X1train,Y1train)
rc_score=rc.score(X1test,Y1test)
svc.fit(X1train,Y1train)
svc_score=svc.score(X1test,Y1test)
bc.fit(X1train,Y1train)
bc_score=bc.score(X1test,Y1test)
gbc.fit(X1train,Y1train)
gbc_score=gbc.score(X1test,Y1test)
rc_score,svc_score,bc_score,gbc_score

(0.7529427324427985,
 0.5452982409734162,
 0.7953974342018252,
 0.8420843803729665)

In [14]:
rf = RandomForestClassifier()
kn = KNeighborsClassifier()
lr = LogisticRegression(max_iter=4000)
rf.fit(X1train,Y1train)
rf_score=rf.score(X1test,Y1test)
kn.fit(X1train,Y1train)
kn_score=kn.score(X1test,Y1test)
lr.fit(X1train,Y1train)
lr_score=lr.score(X1test,Y1test)
rf_score,lr_score,kn_score

(0.805052241766962, 0.8078296521624124, 0.8119296389366486)

In [15]:
n_neighbors = list(range(12,15))
hyperparameters = dict( n_neighbors=n_neighbors)
gs_kn = GridSearchCV(KNeighborsClassifier(),param_grid=hyperparameters,cv=5)
gs_kn.fit(X1train,Y1train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [12, 13, 14]})

In [16]:
gs_kn.score(X1test,Y1test),gs_kn.best_params_

(0.8197328395714852, {'n_neighbors': 13})

In [21]:
lr_grid = {"C": [100, 10, 1.0, 0.1, 0.01],
           "penalty":['none', 'l1', 'l2', 'elasticnet'],
           "solver":['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
          ,"max_iter":[4000]}
gs_lr = GridSearchCV(LogisticRegression(),param_grid=lr_grid,cv=5)
gs_lr.fit(X1train,Y1train)

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\91720\.conda\envs\env\lib\site-package

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\91720\.conda\envs\env\lib\site-package

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 454, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning

...

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\91720\.conda\envs\env\lib\site-package

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.

C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\91720\.conda\envs\env\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver sag supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\91720\.conda\envs\env\lib\site-package

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [100, 10, 1.0, 0.1, 0.01], 'max_iter': [4000],
                         'penalty': ['none', 'l1', 'l2', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']})

In [22]:
gs_lr.best_params_,gs_lr.score(X1test,Y1test)

({'C': 1.0, 'max_iter': 4000, 'penalty': 'l2', 'solver': 'newton-cg'},
 0.8148393069699775)

In [26]:
np.logspace(-4,4,20)

array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04])

In [17]:
rf_grid = {"n_estimators": [100, 200, 500, 1000, 1400]}
gs_rf = GridSearchCV(RandomForestClassifier(),param_grid=rf_grid,cv=5)
gs_rf.fit(X1train,Y1train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 200, 500, 1000, 1400]})

In [19]:
gs_rf.score(X1test,Y1test), gs_rf.best_params_

(0.8065070757836265, {'n_estimators': 1400})

In [25]:
Y1preds=gs_kn.predict(X1test)
print(pd.DataFrame(classification_report(Y1test,Y1preds, output_dict=True)).transpose())

              precision    recall  f1-score      support
0              0.880605  0.775061  0.824469  4130.000000
1              0.763372  0.873506  0.814734  3431.000000
accuracy       0.819733  0.819733  0.819733     0.819733
macro avg      0.821989  0.824283  0.819602  7561.000000
weighted avg   0.827408  0.819733  0.820051  7561.000000


In [21]:
print(confusion_matrix(Y1test,Y1preds))

[[3201  929]
 [ 434 2997]]


In [3]:
X1_new


NameError: name 'X1' is not defined

In [10]:
hgbc_score

0.8395714852532734